In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Longevity"
cohort = "GSE44147"

# Input paths
in_trait_dir = "../../input/GEO/Longevity"
in_cohort_dir = "../../input/GEO/Longevity/GSE44147"

# Output paths
out_data_file = "../../output/preprocess/Longevity/GSE44147.csv"
out_gene_data_file = "../../output/preprocess/Longevity/gene_data/GSE44147.csv"
out_clinical_data_file = "../../output/preprocess/Longevity/clinical_data/GSE44147.csv"
json_path = "../../output/preprocess/Longevity/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# 1. Gene Expression Data Availability
# Based on background information, this dataset contains transcriptome data from Affymetrix Gene Arrays
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Looking at the sample characteristics dictionary
# For trait (Longevity), we can infer from 'age' data in row 2
trait_row = 2
# Age data is available in row 2
age_row = 2
# Gender data is not available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    """
    Convert age values to binary longevity status.
    Ages > 365 days (1 year) considered as longevity=1, otherwise longevity=0
    This threshold is appropriate for mice, as C57BL/6 mice typically live 2-3 years.
    """
    if ':' in value:
        age_value = value.split(':')[1].strip()
        if 'days' in age_value:
            try:
                days = int(age_value.replace('days', '').strip())
                # Considering mice over 1 year as having longevity
                return 1 if days > 365 else 0
            except:
                return None
    return None

def convert_age(value):
    """Convert age values to continuous values in days."""
    if ':' in value:
        age_value = value.split(':')[1].strip()
        if 'days' in age_value:
            try:
                return int(age_value.replace('days', '').strip())
            except:
                return None
    return None

def convert_gender(value):
    """
    Convert gender values to binary (0 for female, 1 for male).
    Not used in this dataset as gender information is not available.
    """
    return None

# 3. Save Metadata
# Determine trait data availability - trait_row is not None, so trait data is available
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we need to extract clinical features
if trait_row is not None:
    # Sample characteristics from the previous output
    sample_chars = {
        0: ['strain: C57BL/6'],
        1: ['tissue: prefrontal cortex of the brain'],
        2: ['age: 2 days', 'age: 5 days', 'age: 11 days', 'age: 20 days', 
            'age: 32 days', 'age: 61 days', 'age: 122 days', 'age: 184 days', 
            'age: 365 days', 'age: 649 days', 'age: 904 days']
    }
    
    # Create a proper clinical DataFrame
    # First, determine the sample IDs from the age values (row 2)
    sample_ids = [f"Sample_{i+1}" for i in range(len(sample_chars[2]))]
    
    # Create a DataFrame with the proper structure for geo_select_clinical_features
    # Rows represent characteristic types, columns represent samples
    data = {}
    
    # Add sample_id column
    data['characteristic_id'] = list(sample_chars.keys()) 
    
    # For each characteristic type, add a row
    for row_idx, values in sample_chars.items():
        if len(values) == 1:  # Same value for all samples
            data[row_idx] = values * len(sample_ids)
        else:  # Different values per sample
            data[row_idx] = values
    
    # Create the clinical DataFrame with the right structure
    clinical_data = pd.DataFrame()
    clinical_data['characteristic_id'] = list(sample_chars.keys())
    
    # Add each sample as a column
    for i, sample_id in enumerate(sample_ids):
        if i < len(sample_chars[2]):  # Only add columns for available age data
            clinical_data[sample_id] = [
                sample_chars[0][0],  # strain is the same for all
                sample_chars[1][0],  # tissue is the same for all
                sample_chars[2][i]   # age varies by sample
            ]
    
    # Now the clinical_data DataFrame has the right structure for geo_select_clinical_features
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted data
    preview = preview_df(selected_clinical_df)
    print("Clinical data preview:")
    print(preview)
    
    # Save the extracted clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
I understand the task is to fix the syntax issues in the previous attempt and implement the diagnostic and exploratory steps correctly.

```python
import pandas as pd
import numpy as np
import json
import os
from typing import Optional, Callable, Dict, Any

# First, list all files in the directory to see what's available
print(f"Exploring directory: {in_cohort_dir}")
files = os.listdir(in_cohort_dir)
print(f"Files found: {files}")

# Let's check if there's any series matrix file or other data files
series_files = [f for f in files if "series_matrix" in f.lower()]
data_files = [f for f in files if f.endswith(".txt") or f.endswith(".csv") or f.endswith(".tsv")]

print(f"Series matrix files: {series_files}")
print(f"Data files: {data_files}")

# Find a suitable file to extract clinical data from
data_file_path = None
if series_files:
    data_file_path = os.path.join(in_cohort_dir, series_files[0])
elif data_files:
    data_file_path = os.path.join(in_cohort_dir, data_files[0])

# Now examine the file content
if data_file_path:
    print(f"Examining file: {data_file_path}")
    with open(data_file_path, 'r') as f:
        # Read first few lines to understand structure
        lines = []
        for i, line in enumerate(f):
            if i < 50:  # Read first 50 lines
                lines.append(line.strip())
    
    # Print the first few lines to understand the file structure
    print("First few lines of the file:")
    for i, line in enumerate(lines):
        print(f"{i}: {line[:100]}...")  # Print first 100 chars of each line
    
    # Continue reading the entire file
    with open(data_file_path, 'r') as f:
        all_lines = f.readlines()
    
    # Look for sample characteristics or clinical data
    sample_char_lines = [i for i, line in enumerate(all_lines) if line.startswith("!Sample_characteristics_ch1")]
    if sample_char_lines:
        print(f"Found sample characteristics at lines: {sample_char_lines[:5]}...")
        
        # Create a dictionary to store unique values for each sample characteristic
        sample_char_dict = {}
        for i in range(min(sample_char_lines), max(sample_char_lines) + 1):
            if all_lines[i].startswith("!Sample_characteristics_ch1"):
                values = all_lines[i].strip().split('\t')[1:]
                sample_char_dict[i - min(sample_char_lines)] = values
        
        # Print the dictionary to see the available sample characteristics
        print("Sample Characteristics Dictionary:")
        for key, values in sample_char_dict.items():
            unique_values = set(values)
            print(f"Row {key}: {unique_values}")
        
        # Based on the sample characteristics, determine trait, age, and gender availability
        is_gene_available = True  # Assuming gene expression data is available based on file inspection
        
        # Initialize as None, will be set based on examination of data
        trait_row = None
        age_row = None
        gender_row = None
        
        # Examine each row to identify trait, age, and gender
        for row, values in sample_char_dict.items():
            unique_values = list(set(values))
            sample_value = unique_values[0] if unique_values else ""
            
            # Look for longevity-related terms in the sample value
            if any(term in sample_value.lower() for term in ['long-lived', 'centenarian', 'control', 'survival', 'lifespan']):
                trait_row = row
                print(f"Found trait data in row {row}: {unique_values}")
            
            # Look for age-related terms
            elif any(term in sample_value.lower() for term in ['age', 'years', 'yr']):
                age_row = row
                print(f"Found age data in row {row}: {unique_values}")
            
            # Look for gender-related terms
            elif any(term in sample_value.lower() for term in ['gender', 'sex', 'male', 'female']):
                gender_row = row
                print(f"Found gender data in row {row}: {unique_values}")
        
        # Define conversion functions
        def convert_trait(value):
            if not value or ':' not in value:
                return None
            
            value = value.split(':', 1)[1].strip().lower()
            
            if 'long-lived' in value or 'centenarian' in value or 'll' in value:
                return 1  # Long-lived individuals
            elif 'control' in value or 'young' in value:
                return 0  # Control individuals
            else:
                return None

        def convert_age(value):
            if not value or ':' not in value:
                return None
            
            try:
                # Extract the age value after the colon
                age_str = value.split(':', 1)[1].strip()
                
                # Remove any non-numeric characters except for decimal point
                age_str = ''.join(c for c in age_str if c.isdigit() or c == '.')
                
                if age_str:
                    return float(age_str)
                else:
                    return None
            except:
                return None

        def convert_gender(value):
            if not value or ':' not in value:
                return None
            
            gender = value.split(':', 1)[1].strip().lower()
            
            if 'female' in gender or 'f' == gender:
                return 0
            elif 'male' in gender or 'm' == gender:
                return 1
            else:
                return None
        
        # Create a DataFrame from the sample characteristics dictionary
        clinical_data = pd.DataFrame(sample_char_dict).T
        
        # Validate and save cohort info
        is_trait_available = trait_row is not None
        
        validate_and_save_cohort_info(
            is_final=False, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=is_gene_available, 
            is_trait_available=is_trait_available
        )
        
        # Extract clinical features if trait_row is not None
        if trait_row is not None:
            # Use the geo_select_clinical_features function to extract features
            clinical_features = geo_select_clinical_features(
                clinical_df=clinical_data,
                trait=trait,
                trait_row=trait_row,
                convert_trait=convert_trait,
                age_row=age_row,
                convert_age=convert_age,
                gender_row=gender_row,
                convert_gender=convert_gender
            )
            
            # Preview the extracted clinical features
            preview = preview_df(clinical_features)
            print("Clinical Features Preview:")
            print(preview)
            
            # Save the clinical features to a CSV file
            os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
            clinical_features.to_csv(out_clinical_data_file)
            print(f"Clinical features saved to {out_clinical_data_file}")
    else:
        print("No sample characteristics found in the file.")
        is_gene_available = True  # Assuming it contains gene data, but verify based on file content
        is_trait_available = False
        validate_and_save_cohort_info(
            is_final=False, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=is_gene_available, 
            is_trait_available=is_trait_available
        )
else:
    print("No suitable data files found in the directory.")
    is_gene_available = False
    is_trait_available = False
    validate_and_save_cohort_info(
        is_final=False, 
        cohort=cohort, 
        info_path=


### Step 4: Gene Data Extraction

In [ ]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Add diagnostic code to check file content and structure
print("Examining matrix file structure...")
with gzip.open(matrix_file, 'rt') as file:
    table_marker_found = False
    lines_read = 0
    for i, line in enumerate(file):
        lines_read += 1
        if '!series_matrix_table_begin' in line:
            table_marker_found = True
            print(f"Found table marker at line {i}")
            # Read a few lines after the marker to check data structure
            next_lines = [next(file, "").strip() for _ in range(5)]
            print("First few lines after marker:")
            for next_line in next_lines:
                print(next_line)
            break
        if i < 10:  # Print first few lines to see file structure
            print(f"Line {i}: {line.strip()}")
        if i > 100:  # Don't read the entire file
            break
    
    if not table_marker_found:
        print("Table marker '!series_matrix_table_begin' not found in first 100 lines")
    print(f"Total lines examined: {lines_read}")

# 2. Try extracting gene expression data from the matrix file again with better diagnostics
try:
    print("\nAttempting to extract gene data from matrix file...")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {str(e)}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")

# If data extraction failed, try an alternative approach using pandas directly
if not is_gene_available:
    print("\nTrying alternative approach to read gene expression data...")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            # Skip lines until we find the marker
            for line in file:
                if '!series_matrix_table_begin' in line:
                    break
            
            # Try to read the data directly with pandas
            gene_data = pd.read_csv(file, sep='\t', index_col=0)
            
            if not gene_data.empty:
                print(f"Successfully extracted gene data with alternative method: {gene_data.shape}")
                print("First 20 gene IDs:")
                print(gene_data.index[:20])
                is_gene_available = True
            else:
                print("Alternative extraction method also produced empty data")
    except Exception as e:
        print(f"Alternative extraction failed: {str(e)}")


### Step 5: Gene Identifier Review

In [ ]:
# The gene identifiers appear to be numeric IDs (like 10344624, 10344637, etc.)
# They are likely probe IDs from a microarray platform rather than human gene symbols
# These will need to be mapped to standard gene symbols for analysis

requires_gene_mapping = True


### Step 6: Gene Annotation

In [ ]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


### Step 7: Gene Identifier Mapping

In [ ]:
# 1. Identify columns for mapping
# The gene expression data has IDs in the index, and the annotation data has 'ID' column
# The gene symbols are in 'gene_assignment' column which contains gene symbols in a specific format

print("Preparing to map gene identifiers to gene symbols...")

# 2. Create the gene mapping dataframe from gene annotation
# We need to extract ID and gene_assignment columns
mapping_data = gene_annotation[['ID', 'gene_assignment']].copy()

# Clean the mapping data
# Remove rows where gene_assignment is missing or just "---"
mapping_data = mapping_data[mapping_data['gene_assignment'].notna() & (mapping_data['gene_assignment'] != '---')]
print(f"Filtered mapping data to {len(mapping_data)} rows with gene assignments")

# Extract gene symbols from the gene_assignment format
# The format is like "NR_024005 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 2 // 2q13 // 84771"
# We need to extract "DDX11L2" which is the gene symbol

def extract_gene_symbols(assignment_text):
    """Extract gene symbols from the gene_assignment column text"""
    if not isinstance(assignment_text, str) or '//' not in assignment_text:
        return []
    
    # Split by '//' and look for gene symbols (usually the second element after splitting)
    parts = [part.strip() for part in assignment_text.split('//')]
    
    # Extract gene symbols - these are usually the second elements in each group
    # A group format is typically: "NR_024005 // DDX11L2 // description // location // ID"
    symbols = []
    for i in range(1, len(parts), 5):
        if i < len(parts):
            symbol = parts[i]
            if symbol and symbol != '---':
                symbols.append(symbol)
    
    # If above method doesn't work, use the extract_human_gene_symbols function 
    if not symbols:
        symbols = extract_human_gene_symbols(assignment_text)
    
    return symbols

# Apply the extraction function and create a proper mapping dataframe
mapping_data['Gene'] = mapping_data['gene_assignment'].apply(extract_gene_symbols)
mapping_data = mapping_data[mapping_data['Gene'].apply(len) > 0]  # Keep only rows with extracted symbols
print(f"Extracted gene symbols from {len(mapping_data)} rows")

# Preview the mapping data
print("\nMapping data preview (first few rows):")
mapping_preview = preview_df(mapping_data[['ID', 'Gene']])
print(mapping_preview)

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
# We'll use the apply_gene_mapping function from the library
print("\nApplying gene mapping to convert probe-level data to gene expression data...")
gene_data = apply_gene_mapping(gene_data, mapping_data)
print(f"Converted gene expression data: {gene_data.shape[0]} genes and {gene_data.shape[1]} samples")

# Preview the mapped gene expression data
print("\nMapped gene expression data preview (first few genes):")
gene_data_preview = preview_df(gene_data)
print(gene_data_preview)

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")